In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=7df7434dda84903eca9906e807e639664ca2c3581bfe77079e72ac2f34f267d0
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
# Importar SparkSession y funciones adicionales
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when


In [ ]:
# Crear una nueva SparkSession
spark = SparkSession.builder \
    .appName("Ventas Demo") \
    .getOrCreate()

In [ ]:
# Extract: Cargar los datos sin procesar de un archivo CSV
ventas_raw_df = spark.read.csv("ventas_db_demo.csv", header=True, inferSchema=True)

# Mostrar los datos crudos
ventas_raw_df.show()


+---------------+------+--------+-----------+-----------+-------------+---------+---------+
|       producto|precio|cantidad|  categoria|fecha_venta|       tienda|  cliente|descuento|
+---------------+------+--------+-----------+-----------+-------------+---------+---------+
|   PlayStation5|   800|       3|Electronica| 2024-01-15|SagaFalabella|Cliente 1|       40|
|           Xbox|   600|       1|       NULL| 2024-03-20|       Ripley|Cliente 2|       70|
|Nintendo Switch|   400|       2|Electronica| 2024-05-10|      Oeschle|Cliente 3|       30|
|     Samsung TV|  1000|       1|Electronica| 2024-02-17|SagaFalabella|Cliente 4|     NULL|
|   Apple iPhone|  1200|       2|Electronica| 2024-06-01|       Ripley|Cliente 5|      100|
|  Lenovo Laptop|   900|       1|Electronica| 2024-07-11|      Oeschle|Cliente 1|       80|
|Sony Headphones|   150|       5|Electronica| 2024-04-22|SagaFalabella|Cliente 2|       10|
|     HP Printer|   250|       3|Electronica| 2024-03-09|       Ripley|Cliente 3

In [ ]:
# Transform: Limpiar y transformar los datos
# Precio puesto en dolares para mostrar como se puede convertir con Spark

ventas_clean_df = ventas_raw_df \
    .fillna({"precio": 0, "descuento": 0, "categoria": "Desconocido"}) \
    .withColumn("precio", col("precio") * 3.8) \
    .withColumn("descuento", col("descuento") * 3.8)

# Calcular total_venta_cliente antes de ajustar el modelo
ventas_clean_df = ventas_clean_df.withColumn("total_compra", (col("precio") * col("cantidad")) - col("descuento"))

# Mostrar los datos después de la limpieza
ventas_clean_df.show()

+---------------+------+--------+-----------+-----------+-------------+---------+---------+------------+
|       producto|precio|cantidad|  categoria|fecha_venta|       tienda|  cliente|descuento|total_compra|
+---------------+------+--------+-----------+-----------+-------------+---------+---------+------------+
|   PlayStation5|3040.0|       3|Electronica| 2024-01-15|SagaFalabella|Cliente 1|    152.0|      8968.0|
|           Xbox|2280.0|       1|Desconocido| 2024-03-20|       Ripley|Cliente 2|    266.0|      2014.0|
|Nintendo Switch|1520.0|       2|Electronica| 2024-05-10|      Oeschle|Cliente 3|    114.0|      2926.0|
|     Samsung TV|3800.0|       1|Electronica| 2024-02-17|SagaFalabella|Cliente 4|      0.0|      3800.0|
|   Apple iPhone|4560.0|       2|Electronica| 2024-06-01|       Ripley|Cliente 5|    380.0|      8740.0|
|  Lenovo Laptop|3420.0|       1|Electronica| 2024-07-11|      Oeschle|Cliente 1|    304.0|      3116.0|
|Sony Headphones| 570.0|       5|Electronica| 2024-04-2

In [ ]:
# Load: Crear una vista temporal para realizar consultas SQL
ventas_clean_df.createOrReplaceTempView("ventas_limpias")


In [ ]:
# Consulta 1: Calcular el total de ventas por producto sin descuento
total_ventas_por_producto = spark.sql("""
    SELECT producto, SUM(precio * cantidad) AS total_ventas_sin_dsc
    FROM ventas_limpias
    GROUP BY producto
    ORDER BY total_ventas_sin_dsc DESC
""")
total_ventas_por_producto.show()

+---------------+--------------------+
|       producto|total_ventas_sin_dsc|
+---------------+--------------------+
|   Apple iPhone|             27360.0|
|     Samsung TV|             22800.0|
|  Lenovo Laptop|             20520.0|
|   PlayStation5|             18240.0|
|           Xbox|             13680.0|
|   Asus Monitor|             11970.0|
|Nintendo Switch|              9120.0|
|Sony Headphones|              8550.0|
|     HP Printer|              5700.0|
|    Razer Mouse|              5320.0|
|   Adidas Shoes|              2736.0|
|    H&M T-shirt|              2508.0|
|   Levi's Jeans|              2394.0|
|       Puma Hat|              2280.0|
|    Nike Jacket|              2280.0|
|Sandalias Crocs|               304.0|
+---------------+--------------------+



In [ ]:
# Consulta 2: Obtener los productos más vendidos (por cantidad)
productos_mas_vendidos = spark.sql("""
    SELECT producto, SUM(cantidad) AS cantidad_vendida
    FROM ventas_limpias
    GROUP BY producto
    ORDER BY cantidad_vendida DESC
""")
productos_mas_vendidos.show()

+---------------+----------------+
|       producto|cantidad_vendida|
+---------------+----------------+
|    H&M T-shirt|              22|
|Sony Headphones|              15|
|    Razer Mouse|              14|
|       Puma Hat|              12|
|   Asus Monitor|               9|
|   Levi's Jeans|               7|
|     Samsung TV|               6|
|Nintendo Switch|               6|
|   PlayStation5|               6|
|  Lenovo Laptop|               6|
|           Xbox|               6|
|     HP Printer|               6|
|   Apple iPhone|               6|
|   Adidas Shoes|               6|
|    Nike Jacket|               3|
|Sandalias Crocs|               2|
+---------------+----------------+



In [ ]:

# Consulta 3: Total de ventas por tienda
ventas_por_categoria = spark.sql("""
    SELECT tienda, SUM(precio * cantidad) AS total_ventas
    FROM ventas_limpias
    GROUP BY tienda
    ORDER BY total_ventas DESC
""")
ventas_por_categoria.show()

+-------------+------------+
|       tienda|total_ventas|
+-------------+------------+
|SagaFalabella|     57304.0|
|       Ripley|     51756.0|
|      Oeschle|     46398.0|
|     PlazaVea|       304.0|
+-------------+------------+



In [ ]:

# Consulta 4: Total de consumo por cliente
consumo_por_cliente = spark.sql("""
    SELECT cliente, SUM(total_compra) AS total_compras_cliente
    FROM ventas_limpias
    GROUP BY cliente
    ORDER BY total_compras_cliente DESC
""")
consumo_por_cliente.show()

+---------+---------------------+
|  cliente|total_compras_cliente|
+---------+---------------------+
|Cliente 1|              40451.0|
|Cliente 4|              36594.0|
|Cliente 5|              34751.0|
|Cliente 2|              23655.0|
|Cliente 3|              16644.0|
+---------+---------------------+



In [ ]:

# Almacenamiento de los datos procesados en formato Parquet (optimizado para grandes volúmenes)
ventas_clean_df.write.mode("overwrite").parquet("/path/to/ventas_limpias_parquet")

# Mostrar el esquema de los datos guardados
ventas_parquet = spark.read.parquet("/path/to/ventas_limpias_parquet")
ventas_parquet.printSchema()

# Generar estadísticas descriptivas sobre los datos
ventas_clean_df.describe().show()

root
 |-- producto: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- cantidad: integer (nullable = true)
 |-- categoria: string (nullable = true)
 |-- fecha_venta: date (nullable = true)
 |-- tienda: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- descuento: double (nullable = true)
 |-- total_compra: double (nullable = true)

+-------+------------+------------------+------------------+-----------+-------------+---------+-----------------+-----------------+
|summary|    producto|            precio|          cantidad|  categoria|       tienda|  cliente|        descuento|     total_compra|
+-------+------------+------------------+------------------+-----------+-------------+---------+-----------------+-----------------+
|  count|          40|                40|                40|         40|           40|       40|               40|               40|
|   mean|        NULL|           1726.15|               3.3|       NULL|         NULL|     NULL|   